In [2]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse

In [84]:
online_link_list = [
    'https://skating-vision.online/events/2021/ryazan0121/',
    'https://skating-vision.online/events/2021/ryazan0321/',
    'https://skating-vision.online/events/2021/ryazan0421/',
    'https://skating-vision.online/events/2021/ryazan0521/',
    'https://skating-vision.online/events/2122/ryazan1021/',
    'https://skating-vision.online/events/2122/ryazan1221/',
    'https://skating-vision.online/events/2122/ryazan0322/',
    'https://skating-vision.online/events/2122/ryazan0422/',
    'https://skating-vision.online/events/2122/ryazan0522/',
    'https://skating-vision.online/events/2223/ryazan1022/',
    'https://skating-vision.online/events/2223/ryazan1222/',
    'https://skating-vision.online/events/2223/ryazan02/',
    'https://skating-vision.online/events/2223/ryazan04/',
    'https://skating-vision.online/events/2223/ryazan05/',
    'https://skating-vision.online/events/2324/ryazan10/',
    'https://skating-vision.online/events/2324/ryazan12/',
    'https://skating-vision.online/events/2324/ryazan03/',
    'https://skating-vision.online/events/2324/ryazan04/',
    'https://result.fspnz.ru/rzn_04_24/',
    'https://skating-vision.online/events/2324/ryazan05/',
    'https://skating-vision.online/events/2425/ryazan10/',
    'https://skating-vision.online/events/2425/ryazan12/',
    'https://result.fspnz.ru/rzn_02_25/',
    'https://skating-vision.online/events/2425/ryazan03/'
]

In [85]:
def get_event_results(online_table_link):
    if online_table_link is None:
        return None
    df = pd.DataFrame(columns=['title','date','place','online_link','category','rank', 'name', 'club', 'points'])
    page = requests.get(online_table_link)
    if(page.encoding == 'ISO-8859-1'):
        page.encoding = 'utf-8'
    soup = BeautifulSoup(page.text, 'html.parser')
    h2s = soup.find_all('h2')
    title = h2s[0].text
    h3s = soup.find_all('h3')
    place = 'Рязань'
    date = datetime(h3s[1].text.split('-')[0].strip(), format='%d.%m.%Y')
    all_res = soup.find_all('a', {'href': re.compile(r'CAT[0-9]{3}RS')})
    qty = 0
    print(date)
    for link in all_res:
        res_link = urllib.parse.urljoin(online_table_link, link.get('href'))
        page = requests.get(res_link)
        if(page.encoding == 'ISO-8859-1'):
            page.encoding = 'utf-8'
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            if len(soup.find_all('h2')) < 2:
                continue
            category = soup.find_all('h2')[1].text
            lines = soup.find_all('tr')
            for line in lines:
                cells = line.find_all('td')
                if len(cells) >= 4:
                    rank = cells[0].text
                    name = cells[1].a.text
                    club = cells[1].text[len(name):]
                    name = ' '.join([x.strip() for x in name.split()])
                    name = name.replace("ё", "е")
                    name = name.replace("Ё", "Е")
                    if len(cells) == 6:
                        points = cells[-3].text
                    else:
                        points = cells[-2].text
                    df.loc[len(df)] = [title, date, place, res_link, category, rank, name, club, points]
                    
    return df

In [86]:
df = pd.DataFrame(columns=['title','date','place','online_link','category','rank', 'name', 'club', 'points'])

for online_link in online_link_list:
    df = pd.concat([df, get_event_results(online_link)])
            

20.01.2021
03.03.2021
12.04.2021
11.05.2021
25.10.2021
09.12.2021
17.03.2022
13.04.2022
19.05.2022
24.10.2022
07.12.2022
09.02.2023
17.04.2023
15.05.2023
31.10.2023
19.12.2023
11.03.2024
16.04.2024
13.04.2024
22.05.2024
22.10.2024
18.12.2024
22.02.2025
18.03.2025


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3441 entries, 0 to 120
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3441 non-null   object
 1   date         3441 non-null   object
 2   place        3441 non-null   object
 3   online_link  3441 non-null   object
 4   category     3441 non-null   object
 5   rank         3441 non-null   object
 6   name         3441 non-null   object
 7   club         3441 non-null   object
 8   points       3441 non-null   object
dtypes: object(9)
memory usage: 268.8+ KB


In [95]:
df.groupby(['date', 'title']).count().sort_values(by='date')

,,place,online_link,category,rank,name,club,points
date,title,,,,,,,
03.03.2021,Областные соревнования «Звёздный лёд»,172,172,172,172,172,172,172
07.12.2022,Первенство Рязанской области,156,156,156,156,156,156,156
09.02.2023,Областные соревнования «Звёздный лёд»,130,130,130,130,130,130,130
09.12.2021,Первенство Рязанской области (финал),155,155,155,155,155,155,155
11.03.2024,Областные соревнования «Звёздный лёд»,111,111,111,111,111,111,111
11.05.2021,Личное первенство ГАУ РО «СШ ДС «Олимпийский»,137,137,137,137,137,137,137
12.04.2021,КУБОК РЯЗАНСКОЙ ОБЛАСТИ,97,97,97,97,97,97,97
13.04.2022,Кубок Рязанской области по фигурному катанию,136,136,136,136,136,136,136
13.04.2024,Весенняя капель 2024,199,199,199,199,199,199,199


In [83]:
df.to_csv("ryazan_dump.csv")